In [ ]:
# %%

import pandas as pd

from source.utils import FREQ_DIR, RESULT_DIR, UCS_DIR, confirm_dir
from source.utils.associate import (BINARY_ASSOC_ARGS, add_extra_am,
                                    associate_ucs, confirm_basic_ucs)
from source.utils.associate import convert_ucs_to_csv as ucs2csv
from source.utils.associate import get_associations_csv as init_am, AM_DF_DIR
from source.utils.associate import manipulate_ucs, seek_readable_ucs, adjust_assoc_columns
pd.set_option('display.float_format', '{:,.2f}'.format)

/home/arh234/anaconda3/envs/dev-sanpi/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# %% [markdown]

 set parameters:
 - `UNIT` options:
   - `'Bigr'`
   - `'Adv'`
   - `'Adj'`
 - `PAT_DIR` options:
   - any directory in `results/freq_out/` containing a `ucs_format/` subdir with a `Trig*.tsv` frequency table
   - developed with the following in mind:
     - `POSmirror`
     - `NEGmirror`
     - `ANYmirror` (which has been populated by simple concatenation in the shell)

In [ ]:
UNIT = 'Adv'
# UNIT = 'Bigr' 
# UNIT = 'Adv'
# PAT_DIR = 'POSmirror'
PAT_DIR = 'NEGmirror'
# PAT_DIR = 'ANYmirror'
FRQ_FLOOR = 3
# FRQ_FLOOR = 10
# FRQ_FLOOR = 20
# FRQ_FLOOR = 50
# FRQ_FLOOR = 100
TRIG_TSV = FREQ_DIR.joinpath(
    f'{PAT_DIR}/ucs_format/Trig{UNIT}_frq-thrMIN-7.35f.tsv')
FOCUS = ['f', 'unexpected_f',
         'conservative_log_ratio',
         'am_p1_given2', 'am_p2_given1',
         'am_p1_given2_simple', 'am_p2_given1_simple',
         'am_log_likelihood',
        #  'mutual_information', 'am_odds_ratio_disc', 't_score',
         'N', 'f1', 'f2', 'E11',
         'l1', 'l2']

In [ ]:
# %%

TRIGGER_POLARITY = {
    'positive': {'all',
                 'always',
                 'both',
                 'either',
                 'every',
                 'everybody',
                 'everyone',
                 'everything',
                 'many',
                 'often',
                 'or',
                 'some',
                 'somebody',
                 'someone',
                 'something',
                 'somethings',
                 'sometimes'},
    'negative': {'barely',
                 'hardly',
                 'neither',
                 'never',
                 'no',
                 'nobody',
                 'none',
                 'nor',
                 'nothing',
                 'seldom',
                 'rarely',
                 'scarcely'},

}

In [ ]:
# %%

TRIGGER_QUANT = {
    'existential': {
        'some',
        'somebody',
        'someone',
        'something',
        'somethings',
        'either',
     'or',
        'sometimes'
    },
    'universal': {
        'all', 'every',
        'always',
        'everybody',
        'everyone',
        'both',
        'everything',
    },
    'not_exist': {
        'neither',
        'never',
        'no',
        'nobody',
        'none',
        'nor',
        'nothing',
    },
    'hedged_not_exist': {
        'barely',
        'hardly',
        'rarely',
        'scarcely',
        'seldom',
        'few'
    },
    'hedged_universal':{
     'many',
     'often',
     }
}

In [ ]:
# %%

def invert_set_dict(d: dict):
    return {v: k for k in d for v in d[k]}

# %% [markdown]

## 1. Run `seek_readable_ucs()` to generate consistent output path

In [ ]:
readable = seek_readable_ucs(min_freq=FRQ_FLOOR,
                             ucs_subdir='trigger_eval',
                             contained_counts_path=TRIG_TSV)
print(readable.relative_to(RESULT_DIR))

    > seeking `trigger_eval/NEGmirror/readable/TrigAdv_frq-thrMIN-7.35f_min3x*` frequency data and initial associations...
ucs/trigger_eval/NEGmirror/readable/TrigAdv_frq-thrMIN-7.35f_min3x.rsort-view_am-only.txt


# %% [markdown]

 Snippet of frequencies to be analyzed (`TRIG_TSV`)

In [ ]:
! head -5 {TRIG_TSV} | column -t

43276  nothing  more
25628  never    too
20655  never    more
15893  never    so
12167  never    as


# %% [markdown]

## 2. Run `confirm_basic_ucs()`

In [ ]:
basic_ucs_path = readable.parent.parent.joinpath(
    readable.name.replace('.rsort-view_am-only.txt', '.ds.gz'))
print(f'Creating initial UCS table: `{basic_ucs_path.relative_to(RESULT_DIR)}')

basic_ucs_path = confirm_basic_ucs(
    basic_ucs_path,
    freq_floor=FRQ_FLOOR,
    contained_counts_path=TRIG_TSV)

Creating initial UCS table: `ucs/trigger_eval/NEGmirror/TrigAdv_frq-thrMIN-7.35f_min3x.ds.gz

## Creating initial UCS table...

```
( cat /share/compling/projects/sanpi/results/freq_out/NEGmirror/ucs_format/TrigAdv_frq-thrMIN-7.35f.tsv \ 
  | ucs-make-tables --types --threshold=3 /share/compling/projects/sanpi/results/ucs/trigger_eval/NEGmirror/TrigAdv_frq-thrMIN-7.35f_min3x.ds.gz ) \ 
  && ucs-sort /share/compling/projects/sanpi/results/ucs/trigger_eval/NEGmirror/TrigAdv_frq-thrMIN-7.35f_min3x.ds.gz BY f2- f1- INTO /share/compling/projects/sanpi/results/ucs/trigger_eval/NEGmirror/TrigAdv_frq-thrMIN-7.35f_min3x.ds.gz
== Note ==
    N = total number of tokens/all counts summed
    V = total number of rows/number of unique l1+l2 combinations before filtering to 3+ tokens
+ time to make table → 00:00:00.488
```

Saving initial frequency table in readable .txt format...

```
ucs-print -o /share/compling/projects/sanpi/results/ucs/trigger_eval/NEGmirror/readable/TrigAdv_frq-thrMIN-7.35f_min

# %% [markdown]

 Excerpt of initial UCS table

In [ ]:
init_readable = UCS_DIR.joinpath(
    f'trigger_eval/{PAT_DIR}/readable'
).joinpath(f'{TRIG_TSV.name.replace(".tsv","")}_min{FRQ_FLOOR}x.init.txt')
! head -7 {init_readable}

      l1  l2                      f     f2      f1       N  
--------  ------------------  -----  -----  ------  ------  
   never  more                20655  77678  109723  289776  
 nothing  more                43276  77678   98788  289776  
    none  more                 5508  77678   31021  289776  
     nor  more                  747  77678   15546  289776  
      no  more                 2764  77678    8890  289776  


# %% [markdown]

## 3. Run `associate_ucs()`

In [ ]:
associate_ucs(basic_ucs_path)

transform_ucs_log = f'/share/compling/projects/sanpi/logs/associate/ucs//ucs-{PAT_DIR}_Trig{UNIT}_frq-thrMIN-7-35f_min{FRQ_FLOOR}x*.log'
! head -15 `ls -t1 {transform_ucs_log} | head -1`
! echo '...'
! tail -2 `ls -t1 {transform_ucs_log} | head -1`


Calculating UCS associations...

```
bash /share/compling/projects/sanpi/script/transform_ucs.sh /share/compling/projects/sanpi/results/ucs/trigger_eval/NEGmirror/TrigAdv_frq-thrMIN-7.35f_min3x.ds.gz
> log will be saved to: /share/compling/projects/sanpi/logs/associate/ucs//ucs-NEGmirror_TrigAdv_frq-thrMIN-7-35f_min3x.2024-05-25_2114.log
...
```

+ time elapsed → 00:00:12.869
# Manipulating TrigAdv_frq-thrMIN-7-35f_min3x ucs table
path to this script: /share/compling/projects/sanpi/script/transform_ucs.sh
Sat May 25 21:14:00 EDT 2024
(TMP: /share/compling/projects/sanpi/results/ucs/trigger_eval/NEGmirror/tmp/tmp_NEGmirror-20240525-211400.TrigAdv_frq-thrMIN-7-35f_min3x)
## Initial Contingency Info

DATA SET FILE:  /share/compling/projects/sanpi/results/ucs/trigger_eval/NEGmirror/TrigAdv_frq-thrMIN-7.35f_min3x.ds.gz

# Frequency signatures computed by the ucs-make-tables tool for relational cooccurrences.
# Sample size:  N = 289776 tokens,  V = 5004 pair types.
# A frequency threshold o

# %% [markdown]

### Define dictionary containing relevant vocab sizes
 !!! Warning This is a `#HACK`: \
     Rather than developing a command/code to retrieve the vocab sizes programmatically,
     I simply copied the values given in the log output of `transform_usc.sh`
     for each `PAT_DIR`+`UNIT` combination

 |      |   ANYmirror |   NEGmirror |   POSmirror |
 |:-----|------------:|------------:|------------:|
 | Bigr |     395,338 |      64,644 |     330,694 |
 | Adv  |      23,125 |       5,004 |      18,121 |
 | Adj  |      83,422 |      21,562 |      61,860 |

In [ ]:
VOCABS = {'ANYmirror': {'Adv': 23125, 'Bigr': 395338, 'Adj': 83422},
          'NEGmirror': {'Adv': 5004, 'Bigr': 64644, 'Adj': 21562},
          'POSmirror': {'Adv': 18121, 'Bigr': 330694, 'Adj': 61860}
          }  # HACK
VOCAB = VOCABS[PAT_DIR][UNIT]
pd.DataFrame(VOCABS)

,ANYmirror,NEGmirror,POSmirror
Adv,23125,5004,18121
Bigr,395338,64644,330694
Adj,83422,21562,61860


# %% [markdown]

## 4. Run `ucs_to_csv()` to convert `ucs/[PAT_DIR]/readable/*.txt` to format that `pandas` can parse as a dataframe

In [ ]:
! head -5 {readable}
csv_path = ucs2csv(readable)
print(f'CSV: `{csv_path.relative_to(RESULT_DIR)}`')

      l1  l2                      f           E11  am.log.likelihood  am.odds.ratio.disc   am.p1.given2   am.p2.given1  am.p1.given2.simple  am.p2.given1.simple      f1     f2       N  
--------  ------------------  -----  ------------  -----------------  ------------------  -------------  -------------  -------------------  -------------------  ------  -----  ------  
      no  doubt                  21      0.644256         146.383732         3.134100184   0.9693913824   0.0023622047         1.0000000000         0.0023622047    8890     21  289776  
      no  heavenly                7      0.214752          48.783880         2.676038019   0.9693445469   0.0007874016         1.0000000000         0.0007874016    8890      7  289776  
      no  strings                 5      0.153394          34.844538         2.541241681   0.9693378564   0.0005624297         1.0000000000         0.0005624297    8890      5  289776  
UCS table text converted & saved as /share/compling/projects/sanpi/res

# %% [markdown]

##

In [ ]:
trig_amdf = pd.read_csv(csv_path).convert_dtypes()
trig_amdf

,l1,l2,f,E11,am_log_likelihood,am_odds_ratio_disc,am_p1_given2,am_p2_given1,am_p1_given2_simple,am_p2_given1_simple,f1,f2,N
0,no,doubt,21,0.64,146.38,3.13,0.97,0.00,1.00,0.00,8890,21,289776
1,no,heavenly,7,0.21,48.78,2.68,0.97,0.00,1.00,0.00,8890,7,289776
2,no,strings,5,0.15,34.84,2.54,0.97,0.00,1.00,0.00,8890,5,289776
3,no,t,3,0.09,20.91,2.34,0.97,0.00,1.00,0.00,8890,3,289776
4,nor,alike,4,0.21,23.40,2.20,0.95,0.00,1.00,0.00,15546,4,289776
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1845,never,however,3,38.62,-73.00,-1.24,-0.35,-0.00,0.03,0.00,109723,102,289776
1846,never,structurally,3,38.62,-73.00,-1.24,-0.35,-0.00,0.03,0.00,109723,102,289776
1847,never,necessarily,20,363.50,-740.76,-1.45,-0.36,-0.01,0.02,0.00,109723,960,289776
1848,never,fundamentally,5,121.17,-258.19,-1.54,-0.36,-0.00,0.02,0.00,109723,320,289776


In [ ]:
# %%

trig_amdf['key'] = (trig_amdf.l1 + '~' +
                    trig_amdf.l2).astype('string')
trig_amdf = trig_amdf.set_index('key')
trig_amdf

,l1,l2,f,E11,am_log_likelihood,am_odds_ratio_disc,am_p1_given2,am_p2_given1,am_p1_given2_simple,am_p2_given1_simple,f1,f2,N
key,,,,,,,,,,,,,
no~doubt,no,doubt,21,0.64,146.38,3.13,0.97,0.00,1.00,0.00,8890,21,289776
no~heavenly,no,heavenly,7,0.21,48.78,2.68,0.97,0.00,1.00,0.00,8890,7,289776
no~strings,no,strings,5,0.15,34.84,2.54,0.97,0.00,1.00,0.00,8890,5,289776
no~t,no,t,3,0.09,20.91,2.34,0.97,0.00,1.00,0.00,8890,3,289776
nor~alike,nor,alike,4,0.21,23.40,2.20,0.95,0.00,1.00,0.00,15546,4,289776
...,...,...,...,...,...,...,...,...,...,...,...,...,...
never~however,never,however,3,38.62,-73.00,-1.24,-0.35,-0.00,0.03,0.00,109723,102,289776
never~structurally,never,structurally,3,38.62,-73.00,-1.24,-0.35,-0.00,0.03,0.00,109723,102,289776
never~necessarily,never,necessarily,20,363.50,-740.76,-1.45,-0.36,-0.01,0.02,0.00,109723,960,289776


# %% [markdown]

## 6. Save to `./results/assoc_df/`

In [ ]:
df_csv_path = AM_DF_DIR.joinpath(
    str(csv_path.relative_to(UCS_DIR))
    .replace('/readable', '')
    .replace('.rsort-view_am-only', ''))

if not df_csv_path.is_file():
    confirm_dir(df_csv_path.parent)
    trig_amdf.to_csv(df_csv_path)

df_pkl_path = df_csv_path.with_suffix('.pkl.gz')
if not df_pkl_path.is_file():
    trig_amdf.to_pickle(df_csv_path.with_suffix('.pkl.gz'))

# %% [markdown]

## 7. Add additional AM via `add_extra_am()`

In [ ]:
ex_trig_amdf = add_extra_am(df=trig_amdf,
                            verbose=True,
                            vocab=VOCAB,
                            metrics=['t_score', 'mutual_information']
                            ).convert_dtypes()


Preview of Extended Measures (rounded)

| key           |   t_score |   mutual_information |   deltaP_min |   deltaP_max |   deltaP_max_abs |   deltaP_product |   unexpected_f |   unexpected_ratio |
|:--------------|----------:|---------------------:|-------------:|-------------:|-----------------:|-----------------:|---------------:|-------------------:|
| no~doubt      |      4.44 |                 1.51 |         0.00 |         0.97 |             0.97 |             0.00 |          20.36 |               0.97 |
| no~heavenly   |      2.56 |                 1.51 |         0.00 |         0.97 |             0.97 |             0.00 |           6.79 |               0.97 |
| no~strings    |      2.17 |                 1.51 |         0.00 |         0.97 |             0.97 |             0.00 |           4.85 |               0.97 |
| no~t          |      1.68 |                 1.51 |         0.00 |         0.97 |             0.97 |             0.00 |           2.91 |               0.97 |
| nor

#%% [markdown]

 Add trigger features as columns: polarity and quantification type

In [ ]:
def add_feature(triggers:pd.Series, 
                group_dict: dict) -> pd.Series:
    assign_dict = invert_set_dict(group_dict)
    return triggers.apply(lambda x: assign_dict[x] if x in assign_dict.keys() else '')

ex_trig_amdf['polarity'] = add_feature(ex_trig_amdf.l1, TRIGGER_POLARITY)
ex_trig_amdf['quant'] = add_feature(ex_trig_amdf.l1, TRIGGER_QUANT)

# %% [markdown]

 Save extended AM tables to `extra/` subdirectory if not already saved

In [ ]:
df_extra_csv = df_csv_path.parent / 'extra' / \
    df_csv_path.name.replace('.csv', '_extra.csv')
print(df_extra_csv)
if not df_extra_csv.is_file():
    confirm_dir(df_extra_csv.parent)
    ex_trig_amdf.to_csv(df_extra_csv)

df_extra_pkl = df_extra_csv.with_suffix('.pkl.gz')
if not df_extra_pkl.is_file():
    ex_trig_amdf.to_pickle(df_extra_pkl)

/share/compling/projects/sanpi/results/assoc_df/trigger_eval/NEGmirror/extra/TrigAdv_frq-thrMIN-7.35f_min3x_extra.csv


In [ ]:
# %%

ex_trig_full = ex_trig_amdf.copy()
ex_trig_abbr = adjust_assoc_columns(
    ex_trig_amdf[[c for c in ['polarity', 'quant'] + FOCUS if c in ex_trig_amdf.columns]]).sort_values('LRC', ascending=False)
cols = ex_trig_abbr.columns

# %% [markdown]

 Define lexical items with given lean shown in binary environment evaluation

In [ ]:
pos_prone = {
    'Adj': [
        'unrelated',
        'unable',
        'akin',
        'larger',
        'different',
        'familiar',
        'similar',
        'likely', 
        'brief', 
        'unaware'
    ],
    'Adv': [
        'slightly', 
        'definitely',
        'utterly',
        # LRC top
        'pretty', 
        'rather', 
        'plain', 
        'fairly', 
        'somewhat', 
        'otherwise', 
        'downright',
        'relatively', 
        # G2 top
        # 'very', 
        # 'even', 
        # 'just', 
        # dP1 top (and odds ratio disc)
        'plain', 
        'maybe'
    ], 
    'Bigr': [
        # G2 top
        'completely_different', 
        'too_familiar',
        'even_better',
        # dP1 top
        'quite_different', 
        'too_real', 
        'well_aware', 
        # LRC top
        'too_common', 
        'entirely_different'
        ]}
neg_prone = {
    'Bigr': [
        # LRC top
        'quite_sure',
        'really_sure', 
        'too_early', 
        'too_pleased', 
        'too_fancy',
        # dP1 top
        'entirely_sure', 
        'ever_easy', 
        'ever_perfect', 
        'particularly_surprising',
        'particularly_new',
        # G2 top
        'too_late', 
        'more_important',
        'so_easy', 
        'as_good', 
        'too_old'
        ], 
    'Adv': [
        'yet',
        #LRC top
        'ever',
        'any', 
        'longer', 
        'necessarily',
        'that',
        #dP1 top
        'before', 
        'wise',  #? How is this used as an adverb?
        'earthly',
        'remotely',
        'exactly',
        # G2 top
        'particularly', 
        'too',
        # 'inherently'
        ],
    'Adj': [
        # LRC top
        'early', 
        'late', 
        'fancy', 
        'alone', 
        'sure',
        # dP1 top
        'shabby',
        'demoralizing',
        'alone',
        'aggravating',
        'groundbreaking',
        'eventful',
        #G2 top
        'important', 
        'frustrating',
        'evident', 
        'certain'
        ]
    }
def sort_prone_by_f2(prone_list, amdf):
    return amdf.copy().loc[amdf.l2.isin(prone_list), ['f2','l2']].drop_duplicates().reset_index(drop=True).set_index('l2').round(1).sort_values(['f2'], ascending=False).index.to_list()
pos_prone[UNIT] = sort_prone_by_f2(pos_prone[UNIT], ex_trig_abbr)
neg_prone[UNIT] = sort_prone_by_f2(neg_prone[UNIT], ex_trig_abbr)
pos_prone[UNIT]

['pretty',
 'slightly',
 'utterly',
 'otherwise',
 'relatively',
 'rather',
 'downright',
 'definitely',
 'fairly',
 'somewhat',
 'plain',
 'maybe']

# %% [markdown]

## Strongest associations for each polarity by metric

In [ ]:
def show_metric_top(amdf: pd.DataFrame,
                    metric: str,
                    k=5, cols=[None]):
    if not any(cols):
        cols = amdf.columns
    return (pd.concat((polar_df.nlargest(k, metric)
                       for pol, polar_df in amdf.groupby('polarity')))
            .loc[:, [metric] + cols[cols != metric].to_list()]
            .reset_index(drop=True).set_index(['polarity', 'l1', 'l2'])
            )
def update_prone(exdf, pos_prone, neg_prone) -> dict: 
    prone_dict = {'positive':pos_prone, 
                  'negative': neg_prone}
    for polarity, prone in prone_dict.items():
        try: 
            polar_ex = exdf.loc[polarity, :]
        except KeyError: 
            continue
        else: 
                
            for topw in polar_ex.reset_index().l2.head(2).squeeze(): 
                if topw not in prone[UNIT]: 
                    prone[UNIT].append(topw)
            prone_dict[polarity] = prone
    return prone_dict['positive'], prone_dict['negative']

# %% [markdown]

 Top consevative log ratio $LRC$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr,"LRC")
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

LRC             quant    f  unexp_f   dP1  \
polarity l1       l2                                                       
         nones    religiously 8.08                      5     5.00  0.25   
                  not         0.34                      3     2.84  0.00   
                  more        0.00                      5    -3.85 -0.00   
         nothings more        0.00                      5    -3.04 -0.00   
                  as          0.00                      5     1.78  0.00   
negative no       longer      7.67         not_exist  742   717.40  0.90   
         barely   half        6.73  hedged_not_exist   66    65.45  0.51   
         no       damn        5.77         not_exist  108   104.13  0.83   
                  earthly     5.13         not_exist   74    71.67  0.94   
         barely   above       4.93  hedged_not_exist   12    11.89  0.48   

                                dP2  dP1_simple  dP2_simple       G2       N  \
polarity l1       l2                                                           
         nones    religiously  0.15        0.25        0.15    69.11  289776   
                  not          0.09        0.00        0.09    12.15  289776   
                  more        -0.12        0.00        0.15    -2.57  289776   
         nothings more        -0.10        0.00        0.17    -1.73  289776   
                  as           0.06        0.00        0.17     0.96  289776   
negative no       longer       0.08        0.93        0.08 4,809.26  289776   
         barely   half         0.05        0.52        0.05   547.22  289776   
         no       damn         0.01        0.86        0.01   651.62  289776   
                  earthly      0.01        0.97        0.01   497.88  289776   
         barely   above        0.01        0.48        0.01    96.60  289776   

                                 f1     f2  exp_f  
polarity l1       l2                               
         nones    religiously    33     20   0.00  
                  not            33   1408   0.16  
                  more           33  77678   8.85  
         nothings more           30  77678   8.04  
                  as             30  31129   3.22  
negative no       longer       8890    802  24.60  
         barely   half         1235    128   0.55  
         no       damn         8890    126   3.87  
                  earthly      8890     76   2.33  
         barely   above        1235     25   0.11

# %% [markdown]

 Top $\Delta P(\texttt{trigger}|\texttt{adv})$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'dP1')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

dP1      quant   f  unexp_f  LRC  dP2  \
polarity l1       l2                                                   
         nones    religiously 0.25              5     5.00 8.08 0.15   
                  not         0.00              3     2.84 0.34 0.09   
                  less        0.00              3     2.45 0.00 0.07   
         nothings very        0.00              4     3.08 0.00 0.10   
                  as          0.00              5     1.78 0.00 0.06   
negative no       doubt       0.97  not_exist  21    20.36 2.15 0.00   
                  heavenly    0.97  not_exist   7     6.79 0.24 0.00   
                  strings     0.97  not_exist   5     4.85 0.00 0.00   
                  t           0.97  not_exist   3     2.91 0.00 0.00   
         nor      alike       0.95  not_exist   4     3.79 0.00 0.00   

                               dP1_simple  dP2_simple     G2       N     f1  \
polarity l1       l2                                                          
         nones    religiously        0.25        0.15  69.11  289776     33   
                  not                0.00        0.09  12.15  289776     33   
                  less               0.00        0.09   5.46  289776     33   
         nothings very               0.00        0.13   5.92  289776     30   
                  as                 0.00        0.17   0.96  289776     30   
negative no       doubt              1.00        0.00 146.38  289776   8890   
                  heavenly           1.00        0.00  48.78  289776   8890   
                  strings            1.00        0.00  34.84  289776   8890   
                  t                  1.00        0.00  20.91  289776   8890   
         nor      alike              1.00        0.00  23.40  289776  15546   

                                  f2  exp_f  
polarity l1       l2                         
         nones    religiously     20   0.00  
                  not           1408   0.16  
                  less          4837   0.55  
         nothings very          8904   0.92  
                  as           31129   3.22  
negative no       doubt           21   0.64  
                  heavenly         7   0.21  
                  strings          5   0.15  
                  t                3   0.09  
         nor      alike            4   0.21

# %% [markdown]

 Top conditional probability $P(\texttt{trigger}|\texttt{adv})$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'dP1_simple')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

dP1_simple      quant   f  unexp_f  LRC  \
polarity l1       l2                                                        
         nones    religiously           0.25              5     5.00 8.08   
                  not                   0.00              3     2.84 0.34   
                  less                  0.00              3     2.45 0.00   
         nothings very                  0.00              4     3.08 0.00   
                  as                    0.00              5     1.78 0.00   
negative no       doubt                 1.00  not_exist  21    20.36 2.15   
                  heavenly              1.00  not_exist   7     6.79 0.24   
                  strings               1.00  not_exist   5     4.85 0.00   
         never    astronomically        1.00  not_exist   3     1.86 0.00   
                  gleefully             1.00  not_exist   3     1.86 0.00   

                                  dP1  dP2  dP2_simple     G2       N      f1  \
polarity l1       l2                                                            
         nones    religiously    0.25 0.15        0.15  69.11  289776      33   
                  not            0.00 0.09        0.09  12.15  289776      33   
                  less           0.00 0.07        0.09   5.46  289776      33   
         nothings very           0.00 0.10        0.13   5.92  289776      30   
                  as             0.00 0.06        0.17   0.96  289776      30   
negative no       doubt          0.97 0.00        0.00 146.38  289776    8890   
                  heavenly       0.97 0.00        0.00  48.78  289776    8890   
                  strings        0.97 0.00        0.00  34.84  289776    8890   
         never    astronomically 0.62 0.00        0.00   5.83  289776  109723   
                  gleefully      0.62 0.00        0.00   5.83  289776  109723   

                                     f2  exp_f  
polarity l1       l2                            
         nones    religiously        20   0.00  
                  not              1408   0.16  
                  less             4837   0.55  
         nothings very             8904   0.92  
                  as              31129   3.22  
negative no       doubt              21   0.64  
                  heavenly            7   0.21  
                  strings             5   0.15  
         never    astronomically      3   1.14  
                  gleefully           3   1.14

# %% [markdown]

 Top $\Delta P(\texttt{adv}|\texttt{trigger})$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'dP2')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

dP2             quant      f   unexp_f  LRC  \
polarity l1       l2                                                         
         nones    religiously 0.15                        5      5.00 8.08   
         nothings very        0.10                        4      3.08 0.00   
         nones    not         0.09                        3      2.84 0.34   
                  less        0.07                        3      2.45 0.00   
         nothings as          0.06                        5      1.78 0.00   
negative scarcely less        0.26  hedged_not_exist    216    202.90 3.68   
         nothing  more        0.26         not_exist  43276 16,794.67 1.24   
         barely   even        0.25  hedged_not_exist    324    312.02 4.56   
         no       less        0.24         not_exist   2194  2,045.61 4.52   
         scarcely more        0.18  hedged_not_exist    354    143.57 0.47   

                               dP1  dP1_simple  dP2_simple        G2       N  \
polarity l1       l2                                                           
         nones    religiously 0.25        0.25        0.15     69.11  289776   
         nothings very        0.00        0.00        0.13      5.92  289776   
         nones    not         0.00        0.00        0.09     12.15  289776   
                  less        0.00        0.00        0.09      5.46  289776   
         nothings as          0.00        0.00        0.17      0.96  289776   
negative scarcely less        0.04        0.04        0.28    872.39  289776   
         nothing  more        0.30        0.56        0.44 21,342.27  289776   
         barely   even        0.11        0.12        0.26  1,636.38  289776   
         no       less        0.43        0.45        0.25  9,325.77  289776   
         scarcely more        0.00        0.00        0.45    120.79  289776   

                                  f1     f2     exp_f  
polarity l1       l2                                   
         nones    religiously     33     20      0.00  
         nothings very            30   8904      0.92  
         nones    not             33   1408      0.16  
                  less            33   4837      0.55  
         nothings as              30  31129      3.22  
negative scarcely less           785   4837     13.10  
         nothing  more         98788  77678 26,481.33  
         barely   even          1235   2812     11.98  
         no       less          8890   4837    148.39  
         scarcely more           785  77678    210.43

# %% [markdown]

 Top conditional probability $P(\texttt{adv}|\texttt{trigger})$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'dP2_simple')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

dP2_simple             quant      f   unexp_f  \
polarity l1       l2                                                           
         nothings more               0.17                        5     -3.04   
                  as                 0.17                        5      1.78   
                  too                0.17                        5      0.23   
         nones    religiously        0.15                        5      5.00   
                  more               0.15                        5     -3.85   
negative scarcely more               0.45  hedged_not_exist    354    143.57   
         nothing  more               0.44         not_exist  43276 16,794.67   
         hardly   more               0.36  hedged_not_exist   1944    478.24   
         no       more               0.31         not_exist   2764    380.93   
         scarcely less               0.28  hedged_not_exist    216    202.90   

                               LRC   dP1   dP2  dP1_simple        G2       N  \
polarity l1       l2                                                           
         nothings more        0.00 -0.00 -0.10        0.00     -1.73  289776   
                  as          0.00  0.00  0.06        0.00      0.96  289776   
                  too         0.00  0.00  0.01        0.00      0.01  289776   
         nones    religiously 8.08  0.25  0.15        0.25     69.11  289776   
                  more        0.00 -0.00 -0.12        0.00     -2.57  289776   
negative scarcely more        0.47  0.00  0.18        0.00    120.79  289776   
         nothing  more        1.24  0.30  0.26        0.56 21,342.27  289776   
         hardly   more        0.29  0.01  0.09        0.03    205.06  289776   
         no       more        0.11  0.01  0.04        0.04     83.27  289776   
         scarcely less        3.68  0.04  0.26        0.04    872.39  289776   

                                  f1     f2     exp_f  
polarity l1       l2                                   
         nothings more            30  77678      8.04  
                  as              30  31129      3.22  
                  too             30  46073      4.77  
         nones    religiously     33     20      0.00  
                  more            33  77678      8.85  
negative scarcely more           785  77678    210.43  
         nothing  more         98788  77678 26,481.33  
         hardly   more          5468  77678  1,465.76  
         no       more          8890  77678  2,383.07  
         scarcely less           785   4837     13.10

# %% [markdown]

 Top log-likelihood $G^2$ values

In [ ]:
exdf = show_metric_top(ex_trig_abbr, 'G2')
pos_prone, neg_prone = update_prone(exdf, pos_prone, neg_prone)
exdf

G2      quant      f   unexp_f  LRC  dP1  \
polarity l1       l2                                                            
         nones    religiously     69.11                 5      5.00 8.08 0.25   
                  not             12.15                 3      2.84 0.34 0.00   
         nothings very             5.92                 4      3.08 0.00 0.00   
         nones    less             5.46                 3      2.45 0.00 0.00   
         nothings as               0.96                 5      1.78 0.00 0.00   
negative nothing  more        21,342.27  not_exist  43276 16,794.67 1.24 0.30   
         no       less         9,325.77  not_exist   2194  2,045.61 4.52 0.43   
         never    so           7,430.27  not_exist  15893  6,432.49 1.43 0.28   
                  too          7,131.01  not_exist  25628  8,182.57 0.98 0.21   
         no       longer       4,809.26  not_exist    742    717.40 7.67 0.90   

                               dP2  dP1_simple  dP2_simple       N      f1  \
polarity l1       l2                                                         
         nones    religiously 0.15        0.25        0.15  289776      33   
                  not         0.09        0.00        0.09  289776      33   
         nothings very        0.10        0.00        0.13  289776      30   
         nones    less        0.07        0.00        0.09  289776      33   
         nothings as          0.06        0.00        0.17  289776      30   
negative nothing  more        0.26        0.56        0.44  289776   98788   
         no       less        0.24        0.45        0.25  289776    8890   
         never    so          0.09        0.64        0.14  289776  109723   
                  too         0.12        0.56        0.23  289776  109723   
         no       longer      0.08        0.93        0.08  289776    8890   

                                  f2     exp_f  
polarity l1       l2                            
         nones    religiously     20      0.00  
                  not           1408      0.16  
         nothings very          8904      0.92  
         nones    less          4837      0.55  
         nothings as           31129      3.22  
negative nothing  more         77678 26,481.33  
         no       less          4837    148.39  
         never    so           24985  9,460.51  
                  too          46073 17,445.43  
         no       longer         802     24.60

In [ ]:
# %%

sig_trig_abbr = ex_trig_abbr.loc[ex_trig_abbr.LRC.abs() > 1, :]
def show_prone_ex(amdf, prone_list, list_index):
    try: 
        example = prone[list_index]

    except IndexError: 
        return list_index + 1, f'No {list_index}th entry'
    else:
        print(f'>> {example} <<')
        return list_index+ 1, amdf.filter(regex=f'~{example}$', axis=0).iloc[:10, :]

# %% [markdown]

 ---
 Significant Examples for Items demonstrated positive polarity environment lean overall (Top 10 by $|LRC|>1$)

In [ ]:
prone = pos_prone[UNIT]
prone

['pretty',
 'slightly',
 'utterly',
 'otherwise',
 'relatively',
 'rather',
 'downright',
 'definitely',
 'fairly',
 'somewhat',
 'plain',
 'maybe']

In [ ]:
# %%

ix = 0
ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> pretty <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> slightly <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> utterly <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> otherwise <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
nor~otherwise,negative,not_exist,16,13.80,1.24,0.34,0.00,0.39,0.00,41.53,289776,15546,41,2.20,nor,otherwise


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> relatively <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> rather <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> downright <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> definitely <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> fairly <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> somewhat <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> plain <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,


# %% [markdown]

 ---
## Examples for Items demonstrating *Negative* polarity environment lean overall (Top 10 by $|LRC|>1$)

In [ ]:
prone_list = neg_prone[UNIT]
prone = sig_trig_abbr.copy().loc[sig_trig_abbr.l2.isin(prone_list), ['f2','l2']].drop_duplicates().reset_index(drop=True).set_index('l2').round(1).sort_values(['f2'], ascending=False).index.to_list()
prone

['more',
 'too',
 'particularly',
 'less',
 'ever',
 'that',
 'remotely',
 'any',
 'necessarily',
 'exactly',
 'longer',
 'yet',
 'before',
 'half',
 'earthly',
 'wise',
 'doubt']

In [ ]:
# %%

ix = 0
ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> more <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
nothing~more,negative,not_exist,43276,"16,794.67",1.24,0.30,0.26,0.56,0.44,"21,342.27",289776,98788,77678,"26,481.33",nothing,more
rarely~more,negative,hedged_not_exist,409,-780.93,-1.22,-0.01,-0.18,0.01,0.09,-873.40,289776,4439,77678,"1,189.93",rarely,more
nor~more,negative,not_exist,747,"-3,420.30",-2.29,-0.06,-0.23,0.01,0.05,"-5,426.05",289776,15546,77678,"4,167.30",nor,more
neither~more,negative,not_exist,166,"-1,565.15",-2.87,-0.03,-0.25,0.00,0.03,"-2,864.75",289776,6458,77678,"1,731.15",neither,more


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> too <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
barely~too,negative,hedged_not_exist,44,-152.36,-1.12,-0.00,-0.12,0.00,0.04,-195.02,289776,1235,46073,196.36,barely,too
nobody~too,negative,not_exist,352,-651.42,-1.17,-0.02,-0.11,0.01,0.06,-653.39,289776,6311,46073,"1,003.42",nobody,too
scarcely~too,negative,hedged_not_exist,16,-108.81,-1.22,-0.00,-0.14,0.00,0.02,-169.22,289776,785,46073,124.81,scarcely,too
seldom~too,negative,hedged_not_exist,17,-148.04,-1.59,-0.00,-0.14,0.00,0.02,-243.16,289776,1038,46073,165.04,seldom,too
rarely~too,negative,hedged_not_exist,115,-590.78,-1.99,-0.02,-0.14,0.00,0.03,-862.29,289776,4439,46073,705.78,rarely,too
hardly~too,negative,hedged_not_exist,100,-769.39,-2.44,-0.02,-0.14,0.00,0.02,"-1,243.84",289776,5468,46073,869.39,hardly,too
no~too,negative,not_exist,185,"-1,228.47",-2.46,-0.03,-0.14,0.00,0.02,"-1,936.07",289776,8890,46073,"1,413.47",no,too


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> particularly <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
neither~particularly,negative,not_exist,815,609.21,1.84,0.07,0.10,0.09,0.13,"1,129.80",289776,6458,9234,205.79,neither,particularly
none~particularly,negative,not_exist,2028,"1,039.49",1.06,0.12,0.04,0.22,0.07,"1,013.31",289776,31021,9234,988.51,none,particularly
never~particularly,negative,not_exist,1569,"-1,927.43",-1.38,-0.22,-0.03,0.17,0.01,"-1,981.67",289776,109723,9234,"3,496.43",never,particularly
rarely~particularly,negative,hedged_not_exist,14,-127.45,-1.47,-0.01,-0.03,0.00,0.00,-195.72,289776,4439,9234,141.45,rarely,particularly
hardly~particularly,negative,hedged_not_exist,5,-169.24,-1.99,-0.02,-0.03,0.00,0.00,-311.58,289776,5468,9234,174.24,hardly,particularly
no~particularly,negative,not_exist,19,-264.29,-2.32,-0.03,-0.03,0.00,0.00,-441.92,289776,8890,9234,283.29,no,particularly


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> less <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
no~less,negative,not_exist,2194,"2,045.61",4.52,0.43,0.24,0.45,0.25,"9,325.77",289776,8890,4837,148.39,no,less
scarcely~less,negative,hedged_not_exist,216,202.90,3.68,0.04,0.26,0.04,0.28,872.39,289776,785,4837,13.10,scarcely,less
hardly~less,negative,hedged_not_exist,347,255.73,1.63,0.05,0.05,0.07,0.06,441.99,289776,5468,4837,91.27,hardly,less
nothing~less,negative,not_exist,675,-973.99,-1.38,-0.20,-0.01,0.14,0.01,"-1,028.13",289776,98788,4837,"1,648.99",nothing,less
never~less,negative,not_exist,403,"-1,428.52",-2.38,-0.30,-0.02,0.08,0.00,"-2,258.58",289776,109723,4837,"1,831.52",never,less


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> ever <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
hardly~ever,negative,hedged_not_exist,690,601.54,2.89,0.13,0.11,0.15,0.13,"1,785.17",289776,5468,4688,88.46,hardly,ever
rarely~ever,negative,hedged_not_exist,332,260.19,1.91,0.06,0.06,0.07,0.07,527.26,289776,4439,4688,71.81,rarely,ever
never~ever,negative,not_exist,767,"-1,008.10",-1.36,-0.22,-0.01,0.16,0.01,"-1,058.54",289776,109723,4688,"1,775.10",never,ever
no~ever,negative,not_exist,9,-134.82,-1.69,-0.03,-0.02,0.00,0.00,-225.86,289776,8890,4688,143.82,no,ever


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> that <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
never~that,negative,not_exist,2939,"1,313.47",1.60,0.31,0.02,0.68,0.03,"1,670.67",289776,109723,4293,"1,625.53",never,that
rarely~that,negative,hedged_not_exist,194,128.24,1.10,0.03,0.03,0.05,0.04,171.05,289776,4439,4293,65.76,rarely,that
nothing~that,negative,not_exist,370,"-1,093.53",-2.07,-0.26,-0.02,0.09,0.00,"-1,564.86",289776,98788,4293,"1,463.53",nothing,that


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> remotely <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
nothing~remotely,negative,not_exist,1167,539.38,1.40,0.29,0.01,0.63,0.01,659.65,289776,98788,1841,627.62,nothing,remotely
never~remotely,negative,not_exist,130,-567.09,-2.36,-0.31,-0.01,0.07,0.00,-945.89,289776,109723,1841,697.09,never,remotely


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> any <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
hardly~any,negative,hedged_not_exist,244,223.88,3.44,0.21,0.04,0.23,0.04,831.40,289776,5468,1066,20.12,hardly,any
barely~any,negative,hedged_not_exist,66,61.46,3.07,0.06,0.05,0.06,0.05,237.09,289776,1235,1066,4.54,barely,any
scarcely~any,negative,hedged_not_exist,34,31.11,2.40,0.03,0.04,0.03,0.04,107.63,289776,785,1066,2.89,scarcely,any
nothing~any,negative,not_exist,91,-272.41,-1.70,-0.26,-0.00,0.09,0.00,-388.08,289776,98788,1066,363.41,nothing,any


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> necessarily <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
nor~necessarily,negative,not_exist,264,212.50,2.24,0.22,0.01,0.28,0.02,495.13,289776,15546,960,51.50,nor,necessarily
neither~necessarily,negative,not_exist,96,74.61,1.53,0.08,0.01,0.10,0.01,146.00,289776,6458,960,21.39,neither,necessarily
never~necessarily,negative,not_exist,20,-343.50,-3.25,-0.36,-0.01,0.02,0.00,-740.76,289776,109723,960,363.50,never,necessarily


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> exactly <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
nobody~exactly,negative,not_exist,84,66.34,1.57,0.08,0.01,0.10,0.01,135.72,289776,6311,811,17.66,nobody,exactly
neither~exactly,negative,not_exist,73,54.93,1.26,0.07,0.01,0.09,0.01,98.34,289776,6458,811,18.07,neither,exactly
nothing~exactly,negative,not_exist,65,-211.48,-1.66,-0.26,-0.00,0.08,0.00,-309.85,289776,98788,811,276.48,nothing,exactly


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> longer <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
no~longer,negative,not_exist,742,717.40,7.67,0.90,0.08,0.93,0.08,"4,809.26",289776,8890,802,24.60,no,longer
none~longer,negative,not_exist,10,-75.86,-1.00,-0.09,-0.00,0.01,0.00,-116.68,289776,31021,802,85.86,none,longer
nothing~longer,negative,not_exist,19,-254.41,-2.78,-0.32,-0.00,0.02,0.00,-514.98,289776,98788,802,273.41,nothing,longer


In [ ]:
# %%

ix, exdf = show_prone_ex(sig_trig_abbr, prone, ix)
exdf

>> yet <<


,polarity,quant,f,unexp_f,LRC,dP1,dP2,dP1_simple,dP2_simple,G2,N,f1,f2,exp_f,l1,l2
key,,,,,,,,,,,,,,,,
none~yet,negative,not_exist,99,64.85,1.06,0.20,0.00,0.31,0.00,97.23,289776,31021,319,34.15,none,yet
never~yet,negative,not_exist,13,-107.79,-1.84,-0.34,-0.00,0.04,0.00,-207.98,289776,109723,319,120.79,never,yet
